# Project 1

Goal: Compare runtimes of phylocomR array job on Hoffman2 over different iterations and tree sizes

Workflow: 

1. Build R script for 1 iteration of phylocomR ph_comstruct function
***
2. Test an array job on Hoffman2 with 1000 iterations for a tree size of 100
***
3. Gather output files from test job and sort in python to determine min/max times and runtimes (TODO)
***
4. Make Iterative across multiple tree size/ iteration combinations (TODO) 
***
5. Process all data and create graphs 


 # Step 1: R script 

In order to utilize hoffman resources to run what is an otherwise very slow process, I built a small R script that runs one iteration of the ph_comstruct function from phylocomR. the output is a tibble containing all the output data from one iteration of the function. An additional file containing start and stop times is output. The code shown below can be found in the Bird repository in the single_array_run.R file.


```R

args = commandArgs(trailingOnly=TRUE) # accept the SGE_TASK_ID argument from hoffman2
print(args) #prints the arguments which here should be equivalent to each SGE_TASK_ID
n = as.integer(args[1])
install.packages("phylocomr")
install.packages("ape")
install.packages("tictoc")
library("tictoc")
library(phylocomr)
library(ape)


start_time<-format(Sys.time(), "%H:%M:%S") #had to do this to only get the time. 
sample_tree_cluster<-read.tree(file = "sample_tree_for_cluster.tre")

full_tree_cluster<-read.tree(file = "full_tree_for_cluster.tre")

df_touse_cluster<-data.frame(sample = "species", occurrence = 1, names = sample_tree_cluster$tip.label)
res_sample_cluster<- ph_comstruct(sample = df_touse_cluster, phylo = full_tree_cluster, randomizations = 1)
typeof(res_sample_cluster)
#write.table(res_sample_cluster, file = "sample_phcomstruct_output") #better to have csv maybe, easier to manipulate?
write.csv(res_sample_cluster, file = paste("sample_phcomstruct_output.csv", n)) #this i believe is the output file. I think it's maybe better than table?
end_time<-format(Sys.time(), "%H:%M:%S")

write.table(c(start_time, end_time), file = "table_attempt.txt")


```

# Step 2: Single array job

In order to run 1000 randomizations of the above code to achieve a distribution of expected randomized pd values (for later statistical use), I used qsub to run the above R script 1000 times on Hoffman2. Code for this submission script (as seen below) can be found in submit_array_job.sh


```Bash 

#!/bin/bash
#$ -cwd #uses current working directory
# error = Merged with joblog
#$ -o /u/home/m/mchari/joblog_$JOB_ID/joblog.$JOB_ID.$TASK_ID #creates a file called joblog.jobidnumber.taskidnumber to write to. 
#$ -j y 
#$ -l h_rt=0:30:00,h_data=2G #requests 30 minutes, 2GB of data (per core)
#$ -pe shared 2 #requests 2 cores
# Email address to notify
#$ -M $USER@mail #don't change this line, finds your email in the system 
# Notify when
##$ -m bea #sends you an email (b) when the job begins (e) when job ends (a) when job is aborted (error)
#$ -t 1-5:1 # 1 to 10, with step size of 1

# load the job environment:
. /u/local/Modules/default/init/modules.sh
module load R

echo ${SGE_TASK_ID}

# run julia code
echo Running phylo_code for n = ${SGE_TASK_ID} #prints this quote to joblog.jobidnumber
Rscript single_array_run.R ${SGE_TASK_ID}  > /u/home/m/mchari/joblog_$JOB_ID/output.$JOB_ID.${SGE_TASK_ID}

```


# Step 3: Processing single array job